In [1]:
'''
Example: Unsupervised learning with the ECM algorithm.
Train data is often hard to collect in record linkage or data matching
problems. The Expectation-Conditional Maximisation (ECM) algorithm is the most
well known algorithm for unsupervised data matching. The algorithm preforms
relatively well compared to supervised methods.
'''

'\nExample: Unsupervised learning with the ECM algorithm.\nTrain data is often hard to collect in record linkage or data matching\nproblems. The Expectation-Conditional Maximisation (ECM) algorithm is the most\nwell known algorithm for unsupervised data matching. The algorithm preforms\nrelatively well compared to supervised methods.\n'

In [2]:
from __future__ import print_function

import numpy as np

import recordlinkage as rl
from recordlinkage.index import Block
from recordlinkage.compare import Exact, String

In [3]:
'''
Generated as one data set with 10000 records (5000 originals and 5000 duplicates, with one duplicate per original), 
the originals have been split from the duplicates, into dataset4a.csv (containing the 5000 original records) 
and dataset4b.csv (containing the 5000 duplicate records). 
These two data sets can be used for testing linkage procedures.
'''

from recordlinkage.datasets import load_febrl4

In [4]:
# set logging
rl.logging.set_verbosity(rl.logging.INFO)

In [5]:
# load datasets
print('Loading data...')
dfA, dfB, true_links = load_febrl4(return_links=True)
print(len(dfA), 'records in dataset A')
print(len(dfB), 'records in dataset B')
print(len(true_links), 'links between dataset A and B')

Loading data...
5000 records in dataset A
5000 records in dataset B
5000 links between dataset A and B


In [6]:
# start indexing
print('Build index...')
indexer = rl.Index()
indexer.add(Block('given_name'))
indexer.add(Block('surname'))
indexer.add(Block('soc_sec_id'))
candidate_links = indexer.index(dfA, dfB)
print(len(candidate_links), 'candidate links between dataset A and B')

Build index...
INFO:recordlinkage:indexing - initialize Index class
INFO:recordlinkage:indexing [1/?] - time: 0.97s - pairs: 160170/25000000 - rr: 0.99359
INFO:recordlinkage:indexing [1/?] - time: 0.97s - pairs_total: 160170/25000000 - rr_total: 0.99359
160170 candidate links between dataset A and B


In [7]:
# start comparing
print('Start comparing...')
comparer = rl.Compare()
comparer.add(Exact('given_name', 'given_name', label='given_name'))
comparer.add(String('surname', 'surname', method='jarowinkler',
                    threshold=0.85, label='surname'))
comparer.add(Exact('date_of_birth', 'date_of_birth', label='date_of_birth'))
comparer.add(Exact('suburb', 'suburb', label='suburb'))
comparer.add(Exact('state', 'state', label='state'))
comparer.add(String('address_1', 'address_1', threshold=0.85,
                    label='address_1'))
comparer.add(String('address_2', 'address_2', threshold=0.85,
                    label='address_2'))
features = comparer.compute(candidate_links, dfA, dfB)

print('feature shape', features.shape)

Start comparing...
INFO:recordlinkage:comparing - initialize Compare class
INFO:recordlinkage:comparing [1/?] - time: 3.00s - pairs: 160170
INFO:recordlinkage:comparing [1/?] - time: 3.00s - pairs_total: 160170
feature shape (160170, 7)


In [8]:
# Initialise the Expectation-Conditional Maximisation classifier.
cl = rl.ECMClassifier()
cl.fit(features)

INFO:recordlinkage:Classification - start training ECMClassifier
INFO:recordlinkage:Classification - training computation time: ~0.43s


INFO:recordlinkage:Classification - training computation time: ~0.43s


In [9]:
# Print the parameters that are trained (m, u and p). Note that the estimates
# are very good.
print("p probability P(Match):", cl.p)
print("m probabilities P(x_i=1|Match):", cl.m_probs)
print("u probabilities P(x_i=1|Non-Match):", cl.u_probs)
print("log m probabilities P(x_i=1|Match):", cl.log_m_probs)
print("log u probabilities P(x_i=1|Non-Match):", cl.log_u_probs)
print("log weights of features:", cl.log_weights)
print("weights of features:", cl.weights)

p probability P(Match): 0.030824148676875978
m probabilities P(x_i=1|Match): {'given_name': {0.0: 0.3372769003010178, 1.0: 0.6627230996989829}, 'surname': {0.0: 0.14170218384204047, 1.0: 0.8582978161579603}, 'date_of_birth': {0.0: 0.10554662146680602, 1.0: 0.8944533785331947}, 'suburb': {0.0: 0.25514206560593183, 1.0: 0.744857934394069}, 'state': {0.0: 0.05850536747234717, 1.0: 0.9414946325276534}, 'address_1': {0.0: 0.1391819853081702, 1.0: 0.8608180146918301}, 'address_2': {0.0: 0.3129315907268238, 1.0: 0.6870684092731769}}
u probabilities P(x_i=1|Non-Match): {'given_name': {0.0: 0.5234446495580639, 1.0: 0.4765553504419358}, 'surname': {0.0: 0.47386477635509916, 1.0: 0.5261352236449011}, 'date_of_birth': {0.0: 0.9999678499702871, 1.0: 3.2150029712962494e-05}, 'suburb': {0.0: 0.99908164444832, 1.0: 0.0009183555516796433}, 'state': {0.0: 0.7818520169800142, 1.0: 0.21814798301998578}, 'address_1': {0.0: 0.9994843101558166, 1.0: 0.0005156898441834589}, 'address_2': {0.0: 0.99958209965122

In [10]:
# evaluate the model
links = cl.predict(features)
print("Predicted number of links:", len(links))

INFO:recordlinkage:Classification - predict matches and non-matches


INFO:recordlinkage:Classification - predict matches and non-matches


Predicted number of links: 4923


In [11]:
cm = rl.confusion_matrix(true_links, links, len(candidate_links))
print("Confusion matrix:\n", cm)

Confusion matrix:
 [[  4915     85]
 [     8 155162]]


In [12]:
# compute the F-score for this classification
fscore = rl.fscore(cm)
print('fscore', fscore)
recall = rl.recall(true_links, links)
print('recall', recall)
precision = rl.precision(true_links, links)
print('precision', precision)

fscore 0.9906278343242971
recall 0.983
precision 0.9983749746089783


In [13]:
# Predict the match probability for each pair in the dataset.
probs = cl.prob(features)
print(probs)

INFO:recordlinkage:Classification - compute probabilities


INFO:recordlinkage:Classification - compute probabilities


rec_id_1     rec_id_2      
rec-0-org    rec-0-dup-0       1.000000
             rec-1505-dup-0    0.000003
             rec-1636-dup-0    0.000003
             rec-2074-dup-0    0.000170
             rec-2323-dup-0    0.000001
                                 ...   
rec-999-org  rec-675-dup-0     0.000067
             rec-859-dup-0     0.000003
             rec-91-dup-0      0.000001
             rec-911-dup-0     0.000170
             rec-999-dup-0     1.000000
Length: 160170, dtype: float64
